# PLANEJAMENTO
- OBJETIVO:
    - Efetuar uma breve análise exploratória/descritiva dos dados com a finalidade de responder as questões abaixo
- QUESTIONÁRIO:
    - Qual as descrições de cada variável?
    - Qual a frequência absoluta/relativa/acumulada de cada variável?
    - Quais as medidas centrais de cada variável?
    - Quais as medidas de dispersão de cada variável?
    - Como estão distribuídas cada variável?


# IMPORTS E CONFIGURAÇÕES

In [3]:
# Import de bibliotecas
from zipfile import ZipFile  # Leitura de arquivos .zip
import requests  # Requisição web
from io import BytesIO  # Leitura de arquivo em bytes
import numpy as np  # Manipulação de matrizes
import pandas as pd  # Manipulação de dados
from datetime import date  # Manipulação de datas
import plotly.express as px  # Visualização gráfica
import plotly.graph_objects as go  # Visualização gráfica avançada



# Configurações das bibliotecas
pd.options.display.max_columns = None  # Remoção de limitação de número máximo de colunas
pd.options.display.max_rows = None  # Remoção de limitação de número máximo de linhas
pd.options.display.max_colwidth = None  # Remoção de limitação do tamanho máximo das colunas
pd.options.plotting.backend = 'plotly'  # Definição da biblioteca de plotagem gráfica padrão do pandas

# Informações
print('Data do projeto:', date.today().strftime(format=f'%d/%m/%Y'))  # Data do projeto

Data do projeto: 03/03/2024


# EXTRAÇÃO DOS DADOS

In [4]:
# Acessando a pasta ZIP, diretamente do site da b3, contendo o arquivo excel com os dados
url = 'https://www.b3.com.br/data/files/57/E6/AA/A1/68C7781064456178AC094EA8/ClassifSetorial.zip'
response = requests.get(url=url)

# Leitura do arquivo excel dentro da pasta ZIP
with ZipFile(BytesIO(response.content)) as fold:
    file = fold.namelist()[0]
    with fold.open(file) as file:
        df = pd.read_excel(io=file, skiprows=6)

# TRANSFORMAÇÃO DOS DADOS

In [5]:
# Renomeando as colunas necessárias
df = df.rename(columns={'SETOR ECONÔMICO': 'SETOR', 'SEGMENTO': 'NOME', 'LISTAGEM': 'CÓDIGO', 'Unnamed: 4': 'LISTAGEM'})[1:-18]

# Preenchendo valores nulos da coluna SEGMENTO com os valores da coluna NOME onde a coluna CÓDIGO FOR vazia
df.loc[(df['CÓDIGO'].isnull()), 'SEGMENTO'] = df.loc[(df['CÓDIGO'].isnull()), 'NOME']

# Removendo linhas com todos as variáveis nulas
df = df.dropna(how='all')

# Preenchendo valores nulos da coluna LISTAGEM com 'Sem classificação'
df.LISTAGEM = df.LISTAGEM.fillna(value='AUSENTE')

# Preenchendo valores nulos com base nas células anteriores
df['SETOR'].ffill(inplace=True)
df['SUBSETOR'].ffill(inplace=True)
df['SEGMENTO'].ffill(inplace=True)

# Removendo valores inconsistentes
df = df.loc[(df.CÓDIGO!='CÓDIGO') & (df.CÓDIGO!='LISTAGEM') & (~df.CÓDIGO.isnull())]

# Formatando todos os dados
df = df.apply(lambda x: x.str.strip() if x.dtype == 'object' else x)

# Alterando tipo das variáveis se necessário
df = df.apply(lambda x: x.astype('category') if (x.dtype == 'O') and (x.name not in ['CÓDIGO', 'NOME']) else x)

# Reordenando colunas
df = df[['CÓDIGO', 'NOME', 'SETOR', 'SUBSETOR', 'SEGMENTO', 'LISTAGEM']]

# Reiniciando index
df = df.reset_index(drop=True)

# Criando arquivo parquet para acessos posteriores
df.to_parquet(path='./tickers_por_setor.parquet')

# ANÁLISE EXPLORATÓRIA/DESCRITIVA

## Descrição Geral

In [4]:
# Obtendo informações gerais dos dados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 432 entries, 0 to 431
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   CÓDIGO    432 non-null    object  
 1   NOME      432 non-null    object  
 2   SETOR     432 non-null    category
 3   SUBSETOR  432 non-null    category
 4   SEGMENTO  432 non-null    category
 5   LISTAGEM  432 non-null    category
dtypes: category(4), object(2)
memory usage: 13.4+ KB


In [5]:
# Obtendo informações de performance dos dados
df.memory_usage()



Index        132
CÓDIGO      3456
NOME        3456
SETOR        820
SUBSETOR    1856
SEGMENTO    3200
LISTAGEM     804
dtype: int64

In [6]:
# Pré visualização dos dados
df.head()

,CÓDIGO,NOME,SETOR,SUBSETOR,SEGMENTO,LISTAGEM
0,RRRP,3R PETROLEUM,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
1,CSAN,COSAN,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
2,ENAT,ENAUTA PART,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",NM
3,RPMG,PET MANGUINH,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",AUSENTE
4,PETR,PETROBRAS,"Petróleo, Gás e Biocombustíveis","Petróleo, Gás e Biocombustíveis","Exploração, Refino e Distribuição",N2


In [4]:
df.describe()

,CÓDIGO,NOME,SETOR,SUBSETOR,SEGMENTO,LISTAGEM
count,432,432,432,432,432,432
unique,432,432,11,44,83,9
top,RRRP,3R PETROLEUM,Consumo Cíclico,Energia Elétrica,Energia Elétrica,NM
freq,1,1,91,54,54,194


Descrições gerais dos dados: 

Em termos gerais pode-se dizer que a tabela de dados é composta por 432 registros e 6 variáveis, cujo as variáveis "CÓDIGO" e "NOME" são apenas identificadores e as demais variáveis ("SETOR", "SUBSETOR", "SEGMENTO" e "LISTAGEM") tratam-se de categorias nominais e podem ser analisadas. As variáveis "SETOR", "SUBSETOR" e "SEGMENTO", nesta sequência, são níveis e subníveis exclusivos de uma mesma classificação direcionada as atividades econômicas. Já a variável "LISTAGEM" é independente destas ultimas e referencia a classificação do papel.

Para evitar redundância neste projeto, será desconsiderada a variável "SUBSETOR", uma vez que trabalharemos com a variável "SETOR" que possui menor grau de granularidade e "SEGMENTO" que possui maior grau de granularidade.

## Descrição Variável "SETOR"

In [25]:
# Criação de tabela de frequência absoluta da variável 'setor'
f = df.value_counts(subset=['SETOR'], sort=True, ascending=False).rename("QTD").reset_index()
f.head()

,SETOR,QTD
0,Consumo Cíclico,91
1,Bens Industriais,70
2,Financeiro,69
3,Utilidade Pública,65
4,Consumo não Cíclico,32


### Frequência

In [6]:
# Plot gráfico da frequência absoluta da variável
fig = px.bar(template="plotly_dark",
             title= 'Ranking dos setores com maior quantidade de empresas', 
             data_frame=f, 
             x='QTD', 
             y='SETOR', 
             orientation='h',
             color='SETOR',
             text_auto=True)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='y')
fig.update(layout_showlegend=False)
fig.update_xaxes(title_text='Quantidade')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1958: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(required_grouper, sort=False)  # skip one_group groupers


In [24]:
# Plot gráfico da frequência relativa da variável
fig = px.treemap(data_frame=f,
                 path=[px.Constant('Total'), 'SETOR'],
                 values='QTD',
                 template='plotly_dark',
                 title='Quantidade de empresas por setor')

fig.update_traces(hovertemplate=None, hoverinfo='label+value+percent entry', textinfo='label+value+percent entry')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [8]:
# Criação de tabela de frequência acumulada da variável 'setor'
fac = (df.value_counts(subset='SETOR', normalize=True, sort=True, ascending=True).rename("cumulative").cumsum()*100).round(2)

In [25]:
# Plot gráfico da frequência acumulada da variável
fig = px.bar(data_frame=fac, 
             title='Frequência relativa acumulada por setor em ordem crescente',
             x=fac.index, 
             y=fac.values, 
             color=fac.values, 
             text=fac.values, 
             template='plotly_dark')
fig.update_traces(hovertemplate='%{y:.2f}%', texttemplate='%{text:.1f}%')
fig.update_layout(hovermode='x unified')
fig.update_yaxes(title_text='Frequência acumulada')

fig.show()

### Distribuição

In [10]:
# Plot gráfico da dispersão dos dados
fig = px.box(data_frame=f, template='plotly_dark', x='QTD', points='all', title='Distribuição de frequência')
fig.show()

### Medidas

In [21]:
# Medidas de distribuição
print('Mínimo:', f.QTD.min())
print('Q1:', f.QTD.quantile(.25))
print('Q2:', f.QTD.quantile(.50))
print('Q3:', f.QTD.quantile(.75))
print('Máximo:', f.QTD.max())

# Medidas centrais
print('Média da quantidade de empresas por setor:', round(f.QTD.mean()))
print('Mediana da quantidade de empresas por setor:', f.QTD.median())
print('Moda:', f.QTD[0], '-', f.SETOR[0])

# Medidas de dispersão
print('Amplitude:', f.QTD.max()-f.QTD.min())
print('Desvio padrão:', round(f.QTD.std(), 2))
print('Coeficiente de variação:', round(f.QTD.std()/f.QTD.mean(), 2)*100, '%')

Mínimo: 9
Q1: 14.5
Q2: 31.0
Q3: 67.0
Máximo: 91
Média da quantidade de empresas por setor: 39
Mediana da quantidade de empresas por setor: 31.0
Moda: 91 - Consumo Cíclico
Amplitude: 82
Desvio padrão: 28.97
Coeficiente de variação: 74.0 %


### Considerações

Podemos ver através dos gráficos anteriores, que referenciam a variável "setor", um total de 11 categorias que comportam 432 registros. O setor com o maior número de empresas é o "consumo cíclico", com 91 registro, equivalente à 21% do total, e o menor é "comunicações" com apenas 9 registros, equivalente à 2% do total. 

Dessa forma temos como medidas o valor máximo de 91, o valor mínimo de 9, a média para cada setor com 39, a mediana de 31 e um desvio padrão de 29 registros, resultando num coeficiente de variação de 74% dentro de um conjunto de dados com uma amplitude de 82.

De forma aparente, percebe-se um acumulo da maioria das empresas entre os 4 maiores setores (consumo cíclico, bens industriais, financeiro e utilidades públicas, consecutivamente), que comportam 295 registros de 432, ou em termos relativos, cerca de 68% do total, já os demais setores, os 7 dos 11, comportam 137 registros, em termos relativos, cerca de 32%. 



## Descrição Variável "SEGMENTO"

In [27]:
# Criação de tabela de frequência absoluta da variável 'segmento'
f = df.value_counts(subset=['SEGMENTO'], sort=True, ascending=False ).rename("QTD").reset_index()
f.head()

,SEGMENTO,QTD
0,Energia Elétrica,54
1,Incorporações,28
2,Bancos,22
3,Outros,13
4,Programas e Serviços,13


### Frequência

In [28]:
# Plot gráfico da frequência absoluta da variável
fig = px.bar(template="plotly_dark",
             title= 'Ranking dos segmentos com maior quantidade de empresas', 
             data_frame=f, 
             x='QTD', 
             y='SEGMENTO', 
             orientation='h',
             color='SEGMENTO',
             text_auto=True)
fig.update_traces(hovertemplate=None)
fig.update_layout(hovermode='y')
fig.update(layout_showlegend=False)
fig.update_xaxes(title_text='Quantidade')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1958: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



In [7]:
# Plot gráfico da frequência relativa da variável
fig = px.treemap(data_frame=f,
                 path=[px.Constant('Total'), 'SEGMENTO'],
                 values='QTD',
                 template='plotly_dark',
                 title='Quantidade de empresas por segmento')

fig.update_traces(hovertemplate=None, hoverinfo='label+value+percent entry', textinfo='label+value+percent entry')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  dfg = df.groupby(path[i:]).agg(agg_f)


### Distribuição

In [8]:
# Plot gráfico da dispersão dos dados 2
fig = px.box(data_frame=f, template='plotly_dark', x='QTD', points='all', title='Distribuição de frequência')
fig.show()

### Medidas

In [19]:
# Medidas de distribuição
print('Mínimo:', f.QTD.min())
print('Q1:', f.QTD.quantile(.25))
print('Q2:', f.QTD.quantile(.50))
print('Q3:', f.QTD.quantile(.75))
print('Máximo:', f.QTD.max())

# Medidas centrais
print('Média da quantidade de empresas por segmento:', round(f.QTD.mean()))
print('Mediana da quantidade de empresas por segmento:', f.QTD.median())
print('Moda:', f.QTD[0], '-', f.SETOR[0])

# Medidas de dispersão
print('Amplitude:', f.QTD.max()-f.QTD.min())
print('Desvio padrão:', round(f.QTD.std(), 2))
print('Coeficiente de variação:', round(f.QTD.std()/f.QTD.mean(), 2)*100, '%')

Mínimo: 1
Q1: 1.5
Q2: 3.0
Q3: 6.5
Máximo: 54
Média da quantidade de empresas por segmento: 5
Mediana da quantidade de empresas por segmento: 3.0
Moda: 54 - Utilidade Pública
Amplitude: 53
Desvio padrão: 7.14
Coeficiente de variação: 137.0 %


### Considerações

Podemos ver, também, através dos gráficos anteriores, que referenciam a variável "segmento", um total de 83 categorias que comportam 432 registros. O segmento com o maior número de empresas é o "Energia elétrica", com 54 registros, equivalente à 13% do total, e há 21 segmentos como menores comportando apenas 1 registro cada, equivalente à 0,23% do total cada. 

Dessa forma temos como medidas o valor máximo de 54, o valor mínimo de 1, a média para cada segmento com 5, a mediana de 3 e um desvio padrão de aproximadamente 7 registros, resultando num coeficiente de variação de 134% dentro de um conjunto de dados com uma amplitude de 53.

De forma aparente, percebe-se um acumulo da maioria das empresas entre os 3 maiores segmentos (Energia Elétrica, Incorporações, Bancos, consecutivamente), que comportam 104 registros de 432, ou em termos relativos, cerca de 24% do total, contudo, estes encontram-se fora dos padrões e são considerados outliers. 


## Descrição Variável "LISTAGEM"

### Frequência

### Distribuição

### Medidas

### Considerações

## Relação entre Variáveis

### "SETOR" X "SEGMENTO"

In [43]:

fig = px.treemap(data_frame=f, 
                 path=[px.Constant('Total'), 'SETOR', 'SEGMENTO'], 
                 values='QTD', 
                 template='plotly_dark')
fig.update_traces(hovertemplate=None, hoverinfo='label+value+percent entry', textinfo='label+value+percent entry')
fig.show()

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



#### Considerações

Através dos gráficos setoriais que referenciam a variável 'segmento', acima, podemos dizer que há uma maior concentração nos top 3 segmentos, que comportam 104 dos 432 registos, ou em termos relativos, cerca de 24%
podemos visualizar com maiores detalhes a relação entre as variáveis setor e segmento. Observa-se principalmente os top 4 maiores setores (Consumo cíclico, bens industriais, financeiro e utilidades públicas):
- Consumo cíclico:
- Bens industriais:
- Financeiro:
- Utilidades públicas:
- Consumo não cíclico:
- Materiais básicos:
- Saúde:
- Tecnologia da informação:
- Petróleo, gás e biocombustível:
- Comunicação:
- Outros:

In [22]:
fa = df.value_counts(subset=['LISTAGEM', 'SETOR'], sort=False).rename("count").reset_index()

fig = px.treemap(data_frame=fa, 
                 path=[px.Constant('Total'), 'LISTAGEM', 'SETOR'], 
                 values='count', 
                 template='plotly_dark')
fig.update_traces(hovertemplate=None, hoverinfo='label+value+percent entry', textinfo='label+value+percent entry')
fig.show()


c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.

c:\Users\adria\anaconda3\Lib\site-packages\plotly\express\_core.py:1619: FutureWarning:

The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.



### "LISTAGEM" X "SETOR"

#### Considerações
Podemos ver, através do gráfico de caixa acima, a distribuição quantitativa setorial da b3, que nos demonstra, também, que 4 dos 11 setores possuem uma maior concentração numérica de empresas se comparados aos demais.